![IPL](https://i2.wp.com/content.edupristine.com.s3.amazonaws.com/images/blogs/ipl_inner_img1.png?w=500&ssl=1)
## Batting order of each batsman

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import plotly.express as px
b_by_b = pd.read_csv('../input/ipl-complete-dataset-20082020/IPL Ball-by-Ball 2008-2020.csv')
matches = pd.read_csv('../input/ipl-complete-dataset-20082020/IPL Matches 2008-2020.csv')

Below code creates a dictionary by checking id, inning and batsman as key. Batting order is the value for the key.

A new temporary column in dataframe is created using above 3 fields which is then mapped to populate "in_at" column.

In [ ]:
df=b_by_b[['id','inning','over','ball','batsman_runs']].sort_values(by=['id','inning','over','ball'])

bat_man={}; y=1
for i in df.id.unique():
    for j in range(1,3):
        df=b_by_b.query("id==@i and inning==@j").sort_values(by=['inning','over'])
        for x in range(len(df.index)):
            if (i,j,df.batsman.values[x]) not in bat_man:
                bat_man.update({(i,j,df.batsman.values[x]):y})
                y+=1
        y=1

b_by_b['con'] = tuple(zip(b_by_b.id.values, b_by_b.inning.values, b_by_b.batsman.values))
b_by_b['in_at'] = b_by_b.con.map(bat_man)
b_by_b=b_by_b.drop('con',axis=1)

In [ ]:
# DataFrame re-structured
df=b_by_b[np.hstack((b_by_b.columns.values[:6], b_by_b.columns.values[-1], b_by_b.columns.values[6:-1]))]
df.query("id==335982").sort_values(by=['inning','over','ball']).head()

In [ ]:
# Dataframe & visualization for batsmen position and his score. (Most run scored 50 batsmen)

df=b_by_b.groupby('batsman')['batsman_runs'].sum().sort_values(ascending=False).head(50)
blist=df.index
df=b_by_b.query("batsman in @blist").groupby(['batsman','in_at'])['batsman_runs'].sum().reset_index()
df.head()

In [ ]:
fig = px.scatter(df, x='batsman', y="in_at", color = 'batsman_runs', width=1800, height=400, title='Batsman position & score')
fig.show()

In [ ]:
#Visualization for batsmen position and his strike rate.

df_runs=b_by_b.groupby('batsman')['batsman_runs'].sum().sort_values(ascending=False).head(50)
blist=df_runs.index
df_balls=b_by_b.query("batsman in @blist").groupby(['batsman','in_at'])['id'].count().sort_values(ascending=False).head(50).reset_index()
bat=pd.merge(df_runs, df_balls, left_on='batsman', right_on='batsman')
bat['strike_rate']=(100*bat.batsman_runs/bat.id)

fig = px.scatter(bat, x='batsman', y="in_at", color = 'strike_rate', width=1800, height=400, title='Batsman position & strike rate')
fig.show()